1. Extract: Load the dataset to understand its structure, columns, and types of
data it contains.
2. Transform: Clean and preprocess the data. This may involve handling missing values, removing duplicates, converting data types, normalizing or scaling data, and possibly feature engineering.
3. Load: load data into a SQLite database.

# **1. Extract**

In [1]:
import pandas as pd

# Load the dataset
file_path = 'combined_file_final.csv'
df = pd.read_csv(file_path)

# Display basic information about the dataset
info = df.info()

# Display the first few rows of the dataframe
preview = df.head()

info, preview

<ipython-input-1-afff2180617c>:5: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216558 entries, 0 to 216557
Data columns (total 26 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   Date                                  216558 non-null  object 
 1   Source                                216558 non-null  object 
 2   Site ID                               216558 non-null  int64  
 3   POC                                   216558 non-null  int64  
 4   Daily Mean PM10 Concentration         32409 non-null   float64
 5   UNITS                                 216558 non-null  object 
 6   DAILY_AQI_VALUE                       216558 non-null  object 
 7   Site Name                             207794 non-null  object 
 8   DAILY_OBS_COUNT                       216558 non-null  int64  
 9   PERCENT_COMPLETE                      216558 non-null  float64
 10  AQS_PARAMETER_CODE                    216558 non-null  int64  
 11  

(None,
          Date  Source    Site ID  POC  Daily Mean PM10 Concentration  \
 0  01/01/2024  AirNow  280490020    1                           11.0   
 1  01/02/2024  AirNow  280490020    1                           12.0   
 2  01/03/2024  AirNow  280490020    1                           22.0   
 3  01/04/2024  AirNow  280490020    1                           23.0   
 4  01/05/2024  AirNow  280490020    1                           16.0   
 
       UNITS DAILY_AQI_VALUE      Site Name  DAILY_OBS_COUNT  PERCENT_COMPLETE  \
 0  ug/m3 SC              10  Jackson NCORE                1             100.0   
 1  ug/m3 SC              11  Jackson NCORE                1             100.0   
 2  ug/m3 SC              20  Jackson NCORE                1             100.0   
 3  ug/m3 SC              21  Jackson NCORE                1             100.0   
 4  ug/m3 SC              15  Jackson NCORE                1             100.0   
 
    ...  COUNTY_CODE COUNTY  SITE_LATITUDE SITE_LONGITUDE  

Observations:
* There are columns with significant missing values, such as "Daily Mean PM10
Concentration," "Daily Mean PM2.5 Concentration," "Daily Max 8-hour Ozone Concentration," and others, indicating that not all pollutants were measured at each site or on each day.
* The column "DAILY_AQI_VALUE" has a datatype issue (mixed types), which needs to be addressed.
* Some columns may not be necessary for every analysis, depending on your objectives. For example, detailed location information (latitude and longitude) might be redundant if you're analyzing on a per-county or per-state basis.
* There are columns for different pollutants, which suggests that this dataset could support a wide range of analyses or models focused on air quality.

# **2. Transformation**

In [2]:
# Address missing values: For demonstration, fill missing values for numeric columns with median
# and categorical columns with mode. Specific strategy might change based on analysis needs.
for col in df.select_dtypes(include='number').columns:
    df[col].fillna(df[col].median(), inplace=True)

for col in df.select_dtypes(include='object').columns:
    df[col].fillna(df[col].mode()[0], inplace=True)

# Correcting data types: Convert 'DAILY_AQI_VALUE' to numeric, coercing errors which will be replaced with NaN
df['DAILY_AQI_VALUE'] = pd.to_numeric(df['DAILY_AQI_VALUE'], errors='coerce')
# Filling any new NaN values in 'DAILY_AQI_VALUE' after conversion
df['DAILY_AQI_VALUE'].fillna(df['DAILY_AQI_VALUE'].median(), inplace=True)

# Dropping unnecessary columns: As a demonstration, I'll drop a few columns that might not be necessary for a broad analysis.
# This step should be tailored based on specific analysis requirements.
columns_to_drop = ['Site ID', 'POC', 'AQS_PARAMETER_CODE', 'AQS_PARAMETER_DESC']
df.drop(columns=columns_to_drop, inplace=True)

# Display the new structure of the dataframe and check for improvements
df_info_after = df.info()
df_head_after = df.head()

df_info_after, df_head_after


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216558 entries, 0 to 216557
Data columns (total 22 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   Date                                  216558 non-null  object 
 1   Source                                216558 non-null  object 
 2   Daily Mean PM10 Concentration         216558 non-null  float64
 3   UNITS                                 216558 non-null  object 
 4   DAILY_AQI_VALUE                       216558 non-null  float64
 5   Site Name                             216558 non-null  object 
 6   DAILY_OBS_COUNT                       216558 non-null  int64  
 7   PERCENT_COMPLETE                      216558 non-null  float64
 8   CBSA_CODE                             216558 non-null  float64
 9   CBSA_NAME                             216558 non-null  object 
 10  STATE_CODE                            216558 non-null  int64  
 11  

(None,
          Date  Source  Daily Mean PM10 Concentration     UNITS  \
 0  01/01/2024  AirNow                           11.0  ug/m3 SC   
 1  01/02/2024  AirNow                           12.0  ug/m3 SC   
 2  01/03/2024  AirNow                           22.0  ug/m3 SC   
 3  01/04/2024  AirNow                           23.0  ug/m3 SC   
 4  01/05/2024  AirNow                           16.0  ug/m3 SC   
 
    DAILY_AQI_VALUE      Site Name  DAILY_OBS_COUNT  PERCENT_COMPLETE  \
 0             10.0  Jackson NCORE                1             100.0   
 1             11.0  Jackson NCORE                1             100.0   
 2             20.0  Jackson NCORE                1             100.0   
 3             21.0  Jackson NCORE                1             100.0   
 4             15.0  Jackson NCORE                1             100.0   
 
    CBSA_CODE    CBSA_NAME  ...  COUNTY_CODE COUNTY  SITE_LATITUDE  \
 0    27140.0  Jackson, MS  ...           49  Hinds      32.329111   
 1    271

# **3. Load**

In [15]:
import sqlite3

# Connect to SQLite database (this will create the database if it doesn't exist)
conn = sqlite3.connect('AirNow_database.db')

# Create a table - adjust the SQL statement to match your data structure
# This is a generic template; you should replace `Column1`, `Column2`, etc., with your actual column names and types
create_table_sql = """
CREATE TABLE IF NOT EXISTS air_now_data (
    Date TEXT,
    Source TEXT,
    Site_ID INTEGER,
    POC INTEGER,
    Daily_Mean_PM10_Concentration REAL,
    UNITS TEXT,
    DAILY_AQI_VALUE REAL,
    Site_Name TEXT,
    DAILY_OBS_COUNT INTEGER,
    PERCENT_COMPLETE REAL,
    AQS_PARAMETER_CODE INTEGER,
    AQS_PARAMETER_DESC TEXT,
    CBSA_CODE INTEGER,
    CBSA_NAME TEXT,
    STATE_CODE INTEGER,
    STATE TEXT,
    COUNTY_CODE INTEGER,
    COUNTY TEXT,
    SITE_LATITUDE REAL,
    SITE_LONGITUDE REAL,
    Daily_Mean_PM2_5_Concentration REAL,
    Daily_Max_8_hour_Ozone_Concentration REAL,
    Daily_Max_1_hour_NO2_Concentration REAL,
    Daily_Max_8_hour_CO_Concentration REAL,
    Daily_Mean_Pb_Concentration REAL,
    Daily_Max_1_hour_SO2_Concentration REAL
);
"""
conn.execute(create_table_sql)

# Load data into the database
# Replace `my_table` with your actual table name and adjust columns as necessary
df.columns = df.columns.str.replace(' ', '_', regex=False)
df.columns = df.columns.str.replace('.', '_', regex=False)
df.columns = df.columns.str.replace('-', '_', regex=False)
df.to_sql('air_now_data', conn, if_exists='append', index=False)

# Commit changes and close the connection
conn.commit()
conn.close()

print("Data loaded successfully into SQLite database.")


Data loaded successfully into SQLite database.
